# Osprey usage

#### Osprey is a WPS server that runs stream flow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from rvic.core.config import read_config
from datetime import datetime

# Ensure we are in the working directory with access to the data
if os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../../../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps'
osprey = WPSClient(url)

#### The list of available processes can be diplayed using the ? command (ex. bird?).

In [2]:
# NBVAL_IGNORE_OUTPUT
osprey?

Type:            WPSClient
String form:     <birdy.client.base.WPSClient object at 0x7ff9e4740ac8>
File:            ~/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py
Docstring:      
A Web Processing Service for Climate Data Analysis.

Processes
---------

convert
    A simple conversion utility to provide users with the ability to convert old routing model setups into RVIC parameters.

convolution
    Aggregates the flow contribution from all upstream grid cellsat every timestep lagged according the Impuls Response Functions.

parameters
    Develop impulse response functions using inputs from a configuration file or dictionary

full_rvic
    Run full RVIC process combining Parameters and Convolution modules
Class docstring:
Returns a class where every public method is a WPS process available at
the given url.

Example:
    >>> emu = WPSClient(url='<server url>')
    >>> emu.hello('stranger')
    'Hello stranger'
Init docstring: 
Args:
    url (str): Link to WPS p

Furthermore, help for individual processes can be diplayed using the same command (ex. bird.process?).

In [3]:
# NBVAL_IGNORE_OUTPUT
osprey.convert?

Signature: osprey.convert(convert_config, loglevel='INFO')
Docstring:
A simple conversion utility to provide users with the ability to convert old routing model setups into RVIC parameters.

Parameters
----------
convert_config : string
    Path to input configuration file or input dictionary
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py-0>
Type:      method


In [4]:
# NBVAL_IGNORE_OUTPUT
osprey.parameters?

Signature: osprey.parameters(params_config, np=1, version=True, loglevel='INFO')
Docstring:
Develop impulse response functions using inputs from a configuration file or dictionary

Parameters
----------
params_config : string
    Path to input configuration file or input dictionary
np : integer
    Number of processors used to run job
version : boolean
    Return RVIC version string
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py-2>
Type:      method


In [5]:
# NBVAL_IGNORE_OUTPUT
osprey.convolution?

Signature: osprey.convolution(convolve_config, loglevel='INFO')
Docstring:
Aggregates the flow contribution from all upstream grid cellsat every timestep lagged according the Impuls Response Functions.

Parameters
----------
convolve_config : string
    Path to input configuration file or input dictionary
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py-1>
Type:      method


In [6]:
# NBVAL_IGNORE_OUTPUT
osprey.full_rvic?

Signature:
osprey.full_rvic(
    params_config,
    convolve_config,
    version=True,
    np=1,
    loglevel='INFO',
)
Docstring:
Run full RVIC process combining Parameters and Convolution modules

Parameters
----------
params_config : string
    Path to parameters module's input configuration file or input dictionary
convolve_config : string
    Path to convolution module's input configuration file or input dictionary
version : boolean
    Return RVIC version string
np : integer
    Number of processors used to run job
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/osprey/</home/sangwonl/Desktop/osprey/venv/lib/python3.6/site-packages/birdy/client/base.py-3>
Type:      method


#### Now that we know which process we wish to run, we can use the docstring to ensure we provide the appropriate parameters.

In [7]:
# run convert
convert_output = osprey.convert(
    convert_config = "tests/data/configs/convert_local.cfg"
)

In [8]:
# run parameters
params_output = osprey.parameters(
    params_config = "tests/data/configs/parameters_local.cfg"
)

In [9]:
# run convolution
convolve_output = osprey.convolution(
    convolve_config = "tests/data/configs/convolve_opendap.cfg"
)

In [10]:
# run full_rvic
full_rvic_output = osprey.full_rvic(
    params_config = "tests/data/configs/parameters_local.cfg",
    convolve_config = "tests/data/configs/convolve_opendap.cfg"
)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [11]:
def make_tmp_copy(output):
    output_content = requests.get(output.get()[0]).content

    tmp_copy = NamedTemporaryFile(
        suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=False
    )
    tmp_copy.write(output_content)
    return tmp_copy.name

In [12]:
convert_data = Dataset(make_tmp_copy(convert_output))
convert_meatadata = {
    "title": convert_data.title,
    "year": convert_data.history.split()[5],
    "month": convert_data.history.split()[2],
    "day": convert_data.history.split()[3],
    "RvicPourPointsFile": convert_data.RvicPourPointsFile,
    "RvicUHFile": convert_data.RvicUHFile,
    "RvicFdrFile": convert_data.RvicFdrFile,
    "RvicDomainFile": convert_data.RvicDomainFile,
}
convert_data.source.split("/")[-1]

'gunicorn'

In [13]:
convert_config = read_config("tests/data/configs/convert_local.cfg")
expected_metadata = {
    "title": "RVIC parameter file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d"),
    "RvicPourPointsFile": convert_config["UHS_FILES"]["STATION_FILE"].split("/")[-1],
    "RvicUHFile": "unknown",
    "RvicFdrFile": "unknown",
    "RvicDomainFile": convert_config["DOMAIN"]["FILE_NAME"].split("/")[-1],
}

assert convert_meatadata == expected_metadata

In [14]:
params_data = Dataset(make_tmp_copy(params_output))
params_meatadata = {
    "title": params_data.title,
    "year": params_data.history.split()[5],
    "month": params_data.history.split()[2],
    "day": params_data.history.split()[3],
    "RvicPourPointsFile": params_data.RvicPourPointsFile,
    "RvicUHFile": params_data.RvicUHFile,
    "RvicFdrFile": params_data.RvicFdrFile,
    "RvicDomainFile": params_data.RvicDomainFile,
}

In [15]:
params_config = read_config("tests/data/configs/parameters_local.cfg")
expected_metadata["RvicPourPointsFile"] = params_config["POUR_POINTS"]["FILE_NAME"].split("/")[-1]
expected_metadata["RvicUHFile"] = params_config["UH_BOX"]["FILE_NAME"].split("/")[-1]
expected_metadata["RvicFdrFile"] = params_config["ROUTING"]["FILE_NAME"].split("/")[-1]

assert params_meatadata == expected_metadata

In [16]:
convolve_data = Dataset(make_tmp_copy(convolve_output))
convolve_meatadata = {
    "title": convolve_data.title,
    "year": convolve_data.history.split()[5],
    "month": convolve_data.history.split()[2],
    "day": convolve_data.history.split()[3],
    "RvicPourPointsFile": convolve_data.RvicPourPointsFile,
    "RvicUHFile": convolve_data.RvicUHFile,
    "RvicFdrFile": convolve_data.RvicFdrFile,
    "RvicDomainFile": convolve_data.RvicDomainFile,
}

In [17]:
convolve_config = read_config("tests/data/configs/parameters_opendap.cfg")
expected_metadata["title"] = "RVIC history file"

assert convolve_meatadata == expected_metadata

In [18]:
full_rvic_data = Dataset(make_tmp_copy(full_rvic_output))
full_rvic_meatadata = {
    "title": full_rvic_data.title,
    "year": full_rvic_data.history.split()[5],
    "month": full_rvic_data.history.split()[2],
    "day": full_rvic_data.history.split()[3],
    "RvicPourPointsFile": full_rvic_data.RvicPourPointsFile,
    "RvicUHFile": full_rvic_data.RvicUHFile,
    "RvicFdrFile": full_rvic_data.RvicFdrFile,
    "RvicDomainFile": full_rvic_data.RvicDomainFile,
}

In [19]:
assert full_rvic_meatadata == expected_metadata